In [1]:
import sys
from pathlib import Path
import os 
# sys.path.append('/home/gridsan/vyuan/.local/lib/python3.9/site-packages/')

current_path = Path.cwd()

# Go to top of the root and append
root = current_path.parents[3]
sys.path.append(str(root))

os.chdir(root)

In [2]:
import json

from fairness_cv_project.methods_mscoco.label_free_cbm.src.utils import data_utils
from fairness_cv_project.methods_mscoco.label_free_cbm.src.utils import conceptset_utils

In [3]:
"""
CLASS_SIM_CUTOFF: Concenpts with cos similarity higher than this to any class will be removed
OTHER_SIM_CUTOFF: Concenpts with cos similarity higher than this to another concept will be removed
MAX_LEN: max number of characters in a concept

PRINT_PROB: what percentage of filtered concepts will be printed
"""

CLASS_SIM_CUTOFF = 0.85
OTHER_SIM_CUTOFF = 0.9
MAX_LEN = 30
PRINT_PROB = 1

dataset = "mscoco_single"
device = "cpu"

In [4]:
# CASE 1: You just generated concepts from GPT
# EDIT these to use the initial concept sets you want

save_name = "data/concept_sets/{}_filtered.txt".format(dataset)

with open(root / "data/concept_sets/gpt3_init/gpt3_{}_important.json".format(dataset), "r") as f:
    important_dict = json.load(f)
with open(root / "data/concept_sets/gpt3_init/gpt3_{}_superclass.json".format(dataset), "r") as f:
    superclass_dict = json.load(f)
with open(root / "data/concept_sets/gpt3_init/gpt3_{}_around.json".format(dataset), "r") as f:
    around_dict = json.load(f)
with open(root / "data/concept_sets/gpt3_init/gpt3_{}_job.json".format(dataset), "r") as f:
    job_dict = json.load(f)
    
with open(root / data_utils.LABEL_FILES[dataset], "r") as f:
    classes = f.read().split("\n")
    
concepts = set()

for values in important_dict.values():
    concepts.update(set(values))

for values in superclass_dict.values():
    concepts.update(set(values))
    
for values in around_dict.values():
    concepts.update(set(values))
    
for values in job_dict.values():
   concepts.update(set(values))

print(len(concepts))

228


In [5]:
# CASE 2: Read another concept set, and filter it

save_name = "data/concept_sets/{}_augmented_filtered.txt".format(dataset)

# with open(root / 'data' / 'concept_sets' / 'mscoco_single_augmented.txt') as f:
#     concepts = set(f.read().split("\n"))
    
with open(root / data_utils.LABEL_FILES[dataset], "r") as f:
    classes = f.read().split("\n")
    

In [6]:
concepts = conceptset_utils.remove_too_long(concepts, MAX_LEN, PRINT_PROB)

37 a small, round head on the other side
37 a neck that is narrower than the body
34 decorations or graphics on the top
39 four round, cylindrical shapes (wheels)
39 bindings for attaching to feet or boots
33 can be different shapes and sizes
47 a network of strings stretched across the frame
33 a stringed grid across the middle
32 usually made of glass or plastic
228 219


In [7]:
concepts = conceptset_utils.filter_too_similar_to_cls(concepts, classes, CLASS_SIM_CUTOFF, device, PRINT_PROB)

219
Class:tennis racket - Deleting a tennis racket
Class:chair - Deleting a chair
Class:cell phone - Deleting a cell phone
Class:surfboard - Deleting a surfboard
Class:bottle - Deleting a bottle
Class:horse - Deleting a horse
Class:bed - Deleting a bed
Class:car - Deleting a car
Class:couch - Deleting a couch
210
Class:tennis racket - Concept:racket, sim:0.886 - Deleting racket

Class:tennis racket - Concept:racquet, sim:0.883 - Deleting racquet

Class:chair - Concept:a seat, sim:0.870 - Deleting a seat

Class:cell phone - Concept:a phone, sim:0.898 - Deleting a phone

Class:cell phone - Concept:phone, sim:0.911 - Deleting phone

Class:bottle - Concept:a bottle of water, sim:0.862 - Deleting a bottle of water

Class:bottle - Concept:bottles, sim:0.939 - Deleting bottles

Class:skis - Concept:a ski, sim:0.881 - Deleting a ski

Class:car - Concept:a vehicle, sim:0.880 - Deleting a vehicle

Class:car - Concept:other cars, sim:0.853 - Deleting other cars

Class:car - Concept:vehicle, sim:0

In [8]:
concepts = conceptset_utils.filter_too_similar(concepts, OTHER_SIM_CUTOFF, device, PRINT_PROB)

A ski lift - a ski lift , sim:1.0000 - Deleting a ski lift
A ski resort - a ski resort , sim:1.0000 - Deleting a ski resort
A ski slope - a ski slope , sim:1.0000 - Deleting a ski slope
a backrest - a comfortable backrest , sim:0.9495 - Deleting a comfortable backrest
a board - board , sim:0.9483 - Deleting board
a bottle of beer - a bottle of wine , sim:0.9084 - Deleting a bottle of beer
a conveyer belt - a conveyor belt , sim:0.9851 - Deleting a conveyer belt
a display screen - a screen , sim:0.9184 - Deleting a display screen
a handle on one side - two handles on the sides , sim:0.9174 - Deleting two handles on the sides
a keyboard - a keyboard or touchscreen , sim:0.9005 - Deleting a keyboard or touchscreen
a label - a labels , sim:0.9360 - Deleting a labels
a large, rectangular body - a large, rectangular shape , sim:0.9316 - Deleting a large, rectangular body
a large, rectangular shape - a long, rectangular body , sim:0.9047 - Deleting a long, rectangular body
a large, rectangula

In [9]:
with open(save_name, "w") as f:
    f.write(concepts[0])
    for concept in concepts[1:]:
        f.write("\n" + concept)

In [10]:
save_name

'data/concept_sets/mscoco_single_augmented_filtered.txt'